In [1]:
import os, sys
from google.colab import drive
drive.mount("/content/drive/")
os.chdir("/content/drive/My Drive/chess_app_testing")
sys.path.append("/content/drive/My Drive/chess_app_testing")

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

Mounted at /content/drive/


In [2]:
image_size = (32, 32)
batch_size = 32

datagen_white_fields = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = (0.2,0.8),
        shear_range=0.2,
        rescale=1./255,
        horizontal_flip=False,
        fill_mode='nearest')
datagen_black_fields = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = (0.2,0.8),
        shear_range=0.2,
        rescale=1./255,
        horizontal_flip=False,
        fill_mode='nearest')


In [3]:
train_white_fields = datagen_white_fields.flow_from_directory(
    'data/chesscom/white_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 1,
    shuffle=True  
)
test_white_fields = datagen_white_fields.flow_from_directory(
    'data/chesscom/white_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 2,
    shuffle=True  
)

Found 13 images belonging to 13 classes.
Found 13 images belonging to 13 classes.


In [4]:
train_black_fields = datagen_black_fields.flow_from_directory(
    'data/chesscom/black_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 1,
    shuffle=True  
)
test_black_fields = datagen_black_fields.flow_from_directory(
    'data/chesscom/black_fields',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    seed = 2,
    shuffle=True  
)

Found 13 images belonging to 13 classes.
Found 13 images belonging to 13 classes.


In [5]:
white_model = keras.Sequential(
    [
        keras.Input(shape=(32,32,1)), # 32x32 grayscale
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(13, activation="softmax"), # 13 classes
    ]
)

white_model.summary()

black_model = keras.models.clone_model(white_model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [12]:
white_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
epochs = 140

history = white_model.fit(
    train_white_fields, 
    epochs=epochs,
    batch_size = 128,
    validation_data=test_white_fields
    )
white_model.save_weights('white_model_chesscom.h5') 

Epoch 1/140
1/1 [==============================] - 1s 1s/step - loss: 0.2258 - categorical_accuracy: 0.9231 - val_loss: 0.5946 - val_categorical_accuracy: 0.8462
Epoch 2/140
1/1 [==============================] - 0s 153ms/step - loss: 0.6271 - categorical_accuracy: 0.8462 - val_loss: 0.0736 - val_categorical_accuracy: 1.0000
Epoch 3/140
1/1 [==============================] - 0s 167ms/step - loss: 0.3504 - categorical_accuracy: 0.9231 - val_loss: 0.1454 - val_categorical_accuracy: 1.0000
Epoch 4/140
1/1 [==============================] - 0s 165ms/step - loss: 0.2857 - categorical_accuracy: 0.8462 - val_loss: 0.2843 - val_categorical_accuracy: 0.9231
Epoch 5/140
1/1 [==============================] - 0s 161ms/step - loss: 0.2862 - categorical_accuracy: 0.9231 - val_loss: 0.2897 - val_categorical_accuracy: 0.8462
Epoch 6/140
1/1 [==============================] - 0s 164ms/step - loss: 0.3828 - categorical_accuracy: 0.8462 - val_loss: 0.2694 - val_categorical_accuracy: 0.9231
Epoch 7/140
1

In [13]:
black_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
epochs = 80

history = black_model.fit(
    train_black_fields, 
    epochs=epochs,
    batch_size = 128,
    validation_data=test_black_fields
    )
black_model.save_weights('black_model_chesscom.h5') 

Epoch 1/80
1/1 [==============================] - 1s 973ms/step - loss: 1.2768 - categorical_accuracy: 0.5385 - val_loss: 1.1850 - val_categorical_accuracy: 0.7692
Epoch 2/80
1/1 [==============================] - 0s 159ms/step - loss: 1.1217 - categorical_accuracy: 0.5385 - val_loss: 0.9046 - val_categorical_accuracy: 0.8462
Epoch 3/80
1/1 [==============================] - 0s 170ms/step - loss: 1.1934 - categorical_accuracy: 0.6154 - val_loss: 0.9683 - val_categorical_accuracy: 0.6923
Epoch 4/80
1/1 [==============================] - 0s 169ms/step - loss: 0.9547 - categorical_accuracy: 0.6154 - val_loss: 1.0548 - val_categorical_accuracy: 0.5385
Epoch 5/80
1/1 [==============================] - 0s 162ms/step - loss: 1.1686 - categorical_accuracy: 0.6154 - val_loss: 1.0503 - val_categorical_accuracy: 0.7692
Epoch 6/80
1/1 [==============================] - 0s 162ms/step - loss: 1.4115 - categorical_accuracy: 0.5385 - val_loss: 1.0181 - val_categorical_accuracy: 0.6923
Epoch 7/80
1/1 [

In [14]:
converter_white_model = tf.lite.TFLiteConverter.from_keras_model(white_model)
converter_white_model.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter_white_model.convert()

f = open('white_model_chesscom.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

INFO:tensorflow:Assets written to: /tmp/tmprf6752wm/assets


INFO:tensorflow:Assets written to: /tmp/tmprf6752wm/assets


In [15]:
converter_black_model = tf.lite.TFLiteConverter.from_keras_model(black_model)
converter_black_model.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter_black_model.convert()

f = open('black_model_chesscom.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

INFO:tensorflow:Assets written to: /tmp/tmpnqjk3_y8/assets


INFO:tensorflow:Assets written to: /tmp/tmpnqjk3_y8/assets
